In [ ]:
from pyxtension.streams import stream
import re

file = "aoc-8.ex1.txt"
file = "aoc-8.ex2.txt"

In [ ]:
def to_row(line):
    return [[int(c), False, 0] for c in line]

with open(file) as file:
    lines = [line.rstrip() for line in file]

grid = stream(lines)\
    .map(to_row)\
    .toList()

In [ ]:
def get_width(grid):
    return len(grid[0])

def get_height(grid):
    return len(grid)

def right(coordinate):
    (x, y) = coordinate
    if (x + 1 >= get_width(grid)):
        return None
    return (x+1, y)

def left(coordinate):
    (x, y) = coordinate
    if (x <= 0):
        return None
    return (x-1, y)

def up(coordinate):
    (x, y) = coordinate
    if (y <= 0):
        return None
    return (x, y-1)

def down(coordinate):
    (x, y) = coordinate
    if (y + 1 >= get_height(grid)):
        return None
    return (x, y+1)

def tree_height(coordinate):
     (x, y) = coordinate
     return grid[y][x][0]

def is_visible(coordinate):
     (x, y) = coordinate
     return grid[y][x][1]

def mark_visible(coordinate):
     (x, y) = coordinate
     grid[y][x][1] = True

def get_score(coordinate):
     (x, y) = coordinate
     return grid[y][x][2]

def set_score(coordinate, score):
     (x, y) = coordinate
     grid[y][x][2] = score

### Part 1

In [ ]:
def mark_visible_trees(coor, move, max_height=-1):
    if coor is None:
        return
    next_coor = move(coor)
    height = tree_height(coor)
    if (height > max_height):
        mark_visible(coor)
        mark_visible_trees(next_coor, move, height)
    else:
        mark_visible_trees(next_coor, move, max_height)

def mark_horizontal(y):
    left_coor = (0, y)
    right_coor = (get_width(grid)-1, y)
    mark_visible_trees(left_coor, right)
    mark_visible_trees(right_coor, left)
    return y

stream(range(0, get_height(grid)))\
    .map(mark_horizontal)\
    .size()
    
def mark_vertical(x):
    top_coor = (x, 0)
    bottom_coor = (x, get_height(grid)-1)
    mark_visible_trees(top_coor, down)
    mark_visible_trees(bottom_coor, up)
    return x

stream(range(0, get_width(grid)))\
    .map(mark_vertical)\
    .size()
    
stream(range(0, get_height(grid)))\
    .flatMap(lambda y: grid[y])\
    .filter(lambda c: c[1])\
    .size()


### Part 2

In [ ]:
def get_score_in_direction(coor, move, height):
    next_coor = move(coor)
    if next_coor is None:
        return 0
    next_height = tree_height(next_coor)
    if next_height < height:
        return 1 + get_score_in_direction(next_coor, move, height)
    return 1

def get_score(coor):
    height = tree_height(coor)
    return stream([up, right, down, left])\
        .map(lambda m: get_score_in_direction(coor, m, height))\
        .reduce(lambda x, y: x * y)

stream(range(0, get_height(grid)))\
    .flatMap(lambda y: stream(range(0, get_width(grid))).map(lambda x: (x, y)).toList())\
    .map(get_score)\
    .max()